<a href="https://colab.research.google.com/github/sambit9238/keras_tuner/blob/master/sklearn_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from sklearn import ensemble
from sklearn import metrics
from sklearn import datasets
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from kerastuner.tuners import Sklearn
from kerastuner.oracles import BayesianOptimization
from kerastuner import Objective

In [18]:
X, y = make_classification(n_samples=1000, n_features=10,
                          n_informative=2, n_redundant=0,
                           random_state=42, shuffle=False)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [33]:
def build_tree_model(hp):
  return ensemble.RandomForestClassifier(
      n_estimators=hp.Int('n_estimators', 10, 80, step=10),
      max_depth=hp.Int('max_depth', 5, 10, step=1),
      max_features=hp.Choice('max_features', ['auto', 'sqrt', 'log2'])
  )

In [34]:
sk_tuner = Sklearn(
                oracle=BayesianOptimization(
                    objective = Objective('score', 'max'),
                    max_trials=10,
                    seed=42
                ),
                hypermodel=build_tree_model,
                scoring=metrics.make_scorer(metrics.accuracy_score),
                cv=model_selection.StratifiedKFold(5),
                directory='log_dir',
                project_name='tune_rf'
        )

INFO:tensorflow:Reloading Oracle from existing project log_dir/tune_rf/oracle.json
INFO:tensorflow:Reloading Tuner from log_dir/tune_rf/tuner0.json


In [35]:
sk_tuner.search(X_train, y_train)

INFO:tensorflow:Oracle triggered exit


In [36]:
sk_tuner.get_best_hyperparameters()[0].values

{'max_depth': 7, 'max_features': 'auto', 'n_estimators': 80}